## Data wrangling

CBOE options data comes in a format that is different from the one we are currently using.  
We would like to have it all in a unified, tidy schema, with calls and puts separated into their own rows.

`IMPORTANT`
- Vega and Theta are missing. We could in the future do the calculation and add them to the data set
- There are two **Net** columns (one for calls and one for puts) in the CBOE data that are not present in our historical data. It tracksWe decided to keep them.
- The **quotedate** will be added when we fetch and store a daily snapshot


In [1]:
import pandas as pd

In [2]:
cboe = pd.read_csv("../data/scraped/cboe/SPX_daily/SPX_20190312.csv", skiprows=2)
historical = pd.read_csv("../data/allspx/SPX_2016.csv")

In [3]:
cboe.columns

Index(['Expiration Date', 'Calls', 'Last Sale', 'Net', 'Bid', 'Ask', 'Vol',
       'IV', 'Delta', 'Gamma', 'Open Int', 'Strike', 'Puts', 'Last Sale.1',
       'Net.1', 'Bid.1', 'Ask.1', 'Vol.1', 'IV.1', 'Delta.1', 'Gamma.1',
       'Open Int.1'],
      dtype='object')

In [4]:
cboe.head()

,Expiration Date,Calls,Last Sale,Net,Bid,Ask,Vol,IV,Delta,Gamma,...,Puts,Last Sale.1,Net.1,Bid.1,Ask.1,Vol.1,IV.1,Delta.1,Gamma.1,Open Int.1
0,03/13/2019,SPXW190313C01700000,0.0,0.00,1086.2,1097.5,0,0.0000,0.9975,0.0,...,SPXW190313P01700000,0.05,0.025,0.0,0.05,3,2.5010,-0.0002,0.0,79
1,03/13/2019,SPXW190313C01800000,0.0,0.00,986.9,998.3,0,2.9426,0.9961,0.0,...,SPXW190313P01800000,0.00,0.000,0.0,0.05,0,2.2504,-0.0003,0.0,34
2,03/13/2019,SPXW190313C01850000,0.0,0.00,935.9,947.3,0,0.0000,0.9988,0.0,...,SPXW190313P01850000,0.00,0.000,0.0,0.05,0,2.0922,-0.0003,0.0,19
3,03/13/2019,SPXW190313C01900000,890.0,7.55,886.1,897.3,2,0.0000,0.9980,0.0,...,SPXW190313P01900000,0.00,0.000,0.0,0.05,0,1.9698,-0.0003,0.0,15
4,03/13/2019,SPXW190313C01950000,0.0,0.00,836.2,847.6,0,0.0000,0.9968,0.0,...,SPXW190313P01950000,0.00,0.000,0.0,0.05,0,1.8414,-0.0003,0.0,16


In [5]:
historical.head()

,underlying,underlying_last,exchange,optionroot,optionext,type,expiration,quotedate,strike,last,bid,ask,volume,openinterest,impliedvol,delta,gamma,theta,vega,optionalias
0,SPX,2008.87,*,SPX160115C00300000,NaN,call,01/15/2016,01/04/2016,300,1748.7,1712.6,1716.5,0,23,0.157,1.0,0.0,-1.8585,0.0,SPX160115C00300000
1,SPX,2008.87,*,SPX160115C00400000,NaN,call,01/15/2016,01/04/2016,400,0.0,1612.7,1616.5,0,0,0.157,1.0,0.0,-2.4780,0.0,SPX160115C00400000
2,SPX,2008.87,*,SPX160115C00500000,NaN,call,01/15/2016,01/04/2016,500,1551.7,1512.7,1516.7,0,2281,0.157,1.0,0.0,-3.0975,0.0,SPX160115C00500000
3,SPX,2008.87,*,SPX160115C00600000,NaN,call,01/15/2016,01/04/2016,600,1425.5,1412.5,1416.7,0,15,0.157,1.0,0.0,-3.7169,0.0,SPX160115C00600000
4,SPX,2008.87,*,SPX160115C00700000,NaN,call,01/15/2016,01/04/2016,700,1249.4,1312.7,1316.8,0,14,0.157,1.0,0.0,-4.3364,0.0,SPX160115C00700000


In [6]:
with open("../data/scraped/cboe/SPX_daily/SPX_20190312.csv") as file:
    first_line = file.readline()
    spot_price = float(first_line.split(",")[1])
    
print(spot_price)

2792.02


In [7]:
call_columns = ["Calls", "Expiration Date", "Strike", "Last Sale", "Net", "Bid", "Ask", "Vol", "Open Int", "IV", "Delta", "Gamma"]

In [8]:
calls = cboe[call_columns]
calls.head()

,Calls,Expiration Date,Strike,Last Sale,Net,Bid,Ask,Vol,Open Int,IV,Delta,Gamma
0,SPXW190313C01700000,03/13/2019,1700.0,0.0,0.00,1086.2,1097.5,0,1,0.0000,0.9975,0.0
1,SPXW190313C01800000,03/13/2019,1800.0,0.0,0.00,986.9,998.3,0,0,2.9426,0.9961,0.0
2,SPXW190313C01850000,03/13/2019,1850.0,0.0,0.00,935.9,947.3,0,0,0.0000,0.9988,0.0
3,SPXW190313C01900000,03/13/2019,1900.0,890.0,7.55,886.1,897.3,2,24,0.0000,0.9980,0.0
4,SPXW190313C01950000,03/13/2019,1950.0,0.0,0.00,836.2,847.6,0,0,0.0000,0.9968,0.0


In [9]:
renamed_columns = ["optionroot", "expiration", "strike", "last", "net", "bid",
       "ask", "volume", "openinterest", "impliedvol", "delta", "gamma"]
calls.columns = renamed_columns
calls.insert(loc=0, column="underlying", value="SPX")
calls.insert(loc=1, column="underlying_last", value=spot_price)
calls.insert(loc=2, column="exchange", value="CBOE")
calls.insert(loc=4, column="type", value="call")
calls.head()

,underlying,underlying_last,exchange,optionroot,type,expiration,strike,last,net,bid,ask,volume,openinterest,impliedvol,delta,gamma
0,SPX,2792.02,CBOE,SPXW190313C01700000,call,03/13/2019,1700.0,0.0,0.00,1086.2,1097.5,0,1,0.0000,0.9975,0.0
1,SPX,2792.02,CBOE,SPXW190313C01800000,call,03/13/2019,1800.0,0.0,0.00,986.9,998.3,0,0,2.9426,0.9961,0.0
2,SPX,2792.02,CBOE,SPXW190313C01850000,call,03/13/2019,1850.0,0.0,0.00,935.9,947.3,0,0,0.0000,0.9988,0.0
3,SPX,2792.02,CBOE,SPXW190313C01900000,call,03/13/2019,1900.0,890.0,7.55,886.1,897.3,2,24,0.0000,0.9980,0.0
4,SPX,2792.02,CBOE,SPXW190313C01950000,call,03/13/2019,1950.0,0.0,0.00,836.2,847.6,0,0,0.0000,0.9968,0.0


In [10]:
put_columns = ["Puts", "Expiration Date", "Strike", "Last Sale.1", "Net.1", "Bid.1", "Ask.1", "Vol.1", "Open Int.1", "IV.1", "Delta.1", "Gamma.1"]

In [11]:
puts = cboe[put_columns]
puts.head()

,Puts,Expiration Date,Strike,Last Sale.1,Net.1,Bid.1,Ask.1,Vol.1,Open Int.1,IV.1,Delta.1,Gamma.1
0,SPXW190313P01700000,03/13/2019,1700.0,0.05,0.025,0.0,0.05,3,79,2.5010,-0.0002,0.0
1,SPXW190313P01800000,03/13/2019,1800.0,0.00,0.000,0.0,0.05,0,34,2.2504,-0.0003,0.0
2,SPXW190313P01850000,03/13/2019,1850.0,0.00,0.000,0.0,0.05,0,19,2.0922,-0.0003,0.0
3,SPXW190313P01900000,03/13/2019,1900.0,0.00,0.000,0.0,0.05,0,15,1.9698,-0.0003,0.0
4,SPXW190313P01950000,03/13/2019,1950.0,0.00,0.000,0.0,0.05,0,16,1.8414,-0.0003,0.0


In [12]:
puts.columns = renamed_columns
puts.insert(loc=0, column="underlying", value="SPX")
puts.insert(loc=1, column="underlying_last", value=spot_price)
puts.insert(loc=2, column="exchange", value="CBOE")
puts.insert(loc=4, column="type", value="put")
puts.head()

,underlying,underlying_last,exchange,optionroot,type,expiration,strike,last,net,bid,ask,volume,openinterest,impliedvol,delta,gamma
0,SPX,2792.02,CBOE,SPXW190313P01700000,put,03/13/2019,1700.0,0.05,0.025,0.0,0.05,3,79,2.5010,-0.0002,0.0
1,SPX,2792.02,CBOE,SPXW190313P01800000,put,03/13/2019,1800.0,0.00,0.000,0.0,0.05,0,34,2.2504,-0.0003,0.0
2,SPX,2792.02,CBOE,SPXW190313P01850000,put,03/13/2019,1850.0,0.00,0.000,0.0,0.05,0,19,2.0922,-0.0003,0.0
3,SPX,2792.02,CBOE,SPXW190313P01900000,put,03/13/2019,1900.0,0.00,0.000,0.0,0.05,0,15,1.9698,-0.0003,0.0
4,SPX,2792.02,CBOE,SPXW190313P01950000,put,03/13/2019,1950.0,0.00,0.000,0.0,0.05,0,16,1.8414,-0.0003,0.0


In [13]:
print("Number of calls:", len(calls))
print("Number of puts:", len(puts))

Number of calls: 6813
Number of puts: 6813


In [14]:
merged = pd.concat([calls, puts])
merged.sample(10)

,underlying,underlying_last,exchange,optionroot,type,expiration,strike,last,net,bid,ask,volume,openinterest,impliedvol,delta,gamma
1648,SPX,2792.02,CBOE,SPXW190329P02395000,put,03/29/2019,2395.0,0.00,0.000,0.35,0.45,0,271,0.2911,-0.0068,0.0001
2296,SPX,2792.02,CBOE,SPXW190408C02720000,call,04/08/2019,2720.0,0.00,0.000,87.40,88.90,0,0,0.1369,0.7681,0.0029
6014,SPX,2792.02,CBOE,SPXW190930P01550000,put,09/30/2019,1550.0,0.00,0.000,1.50,1.70,0,513,0.3379,-0.0065,0.0000
724,SPX,2792.02,CBOE,SPX190315P03075000,put,03/15/2019,3075.0,0.00,0.000,281.80,284.20,0,250,0.0000,-0.9996,0.0000
107,SPX,2792.02,CBOE,SPXW190313C02840000,call,03/13/2019,2840.0,0.20,0.075,0.15,0.25,237,621,0.1471,0.0227,0.0023
2448,SPX,2792.02,CBOE,SPXW190412C01200000,call,04/12/2019,1200.0,0.00,0.000,1585.20,1596.50,0,0,0.9762,0.9970,0.0000
2924,SPX,2792.02,CBOE,SPX190418P02315000,put,04/18/2019,2315.0,0.00,0.000,1.35,1.50,0,69,0.2797,-0.0156,0.0002
359,SPX,2792.02,CBOE,SPXW190315P02140000,put,03/15/2019,2140.0,0.00,0.000,0.00,0.05,0,245,0.8563,-0.0004,0.0000
2137,SPX,2792.02,CBOE,SPXW190405P02605000,put,04/05/2019,2605.0,4.00,-1.550,4.10,4.30,2,464,0.1868,-0.0694,0.0010
2960,SPX,2792.02,CBOE,SPX190418P02405000,put,04/18/2019,2405.0,2.15,-0.475,2.10,2.20,210,63,0.2455,-0.0252,0.0003


In [15]:
assert len(merged) == len(calls) + len(puts)
print("Total option count:", len(merged))

Total option count: 13626
